<a href="https://colab.research.google.com/github/mayatallo/defense-peptides-project/blob/main/JAX_2026_PDDP_immune_Maya_%26_Wimeth_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas
!pip install geopandas --upgrade
import geopandas as gpd

In [2]:
# typical Data Cleaning stack
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler,OrdinalEncoder,OneHotEncoder
from sklearn.impute import SimpleImputer
import requests

In [ ]:
# FOR BLOOD
# Python package to add geneID to our data
import pandas as pd
import requests
import time
from typing import Dict, List

#removing invalid Ids
def clean_protein_ids(idlist: List[str]) -> List[str]:
    """Remove invalid IDs and get unique set"""
    valid_ids = []
    for uid in idlist:
        if uid and len(uid) >= 6 and uid[0].isalpha() and uid != 'heavy':
            valid_ids.append(uid)

    unique_ids = list(set(valid_ids))
    print(f"Cleaned: {len(idlist)} total -> {len(unique_ids)} unique valid IDs")
    return unique_ids

#query uniprot, ask for whole list, give accession numbers, gene ID, gene names
def map_uniprot_simple(idlist: List[str], batch_size: int = 50) -> Dict[str, str]:
    """
    Simplified mapper using direct UniProt entry retrieval.
    """
    mapping = {}
    unique_ids = clean_protein_ids(idlist)

    print(f"\nMapping {len(unique_ids)} unique UniProt IDs...")

    for i in range(0, len(unique_ids), batch_size):
        batch = unique_ids[i:i+batch_size]
        batch_num = i // batch_size + 1
        total_batches = (len(unique_ids) + batch_size - 1) // batch_size

        print(f"Processing batch {batch_num}/{total_batches} ({len(batch)} IDs)...")

        url = "https://rest.uniprot.org/uniprotkb/accessions"
        params = {
            'accessions': ','.join(batch),
            'format': 'json',
            'fields': 'accession,gene_primary'
        }

        try:
            response = requests.get(url, params=params, timeout=30)

            if response.status_code == 200:
                data = response.json()

                if 'results' in data:
                    for entry in data['results']:
                        accession = entry.get('primaryAccession')

                        gene_name = None
                        if 'genes' in entry and entry['genes']:
                            gene_info = entry['genes'][0]
                            if 'geneName' in gene_info:
                                gene_name = gene_info['geneName'].get('value')

                        if accession and gene_name:
                            mapping[accession] = gene_name

                print(f"  Mapped {len([b for b in batch if b in mapping])}/{len(batch)} from this batch")
            else:
                print(f"  Batch failed with status {response.status_code}")

            time.sleep(0.3)

        except Exception as e:
            print(f"  Error in batch {batch_num}: {e}")
            continue

    print(f"\nTotal mapped: {len(mapping)}/{len(unique_ids)}")
    return mapping

#dataframe map
def create_mapped_dataframe(df: pd.DataFrame,
                           protein_id_col: str = 'Protein_id',
                           score_col: str = 'MAPP_absoluteScore') -> pd.DataFrame:
    """Create a new dataframe with gene names added."""
    idlist = df[protein_id_col].tolist()
    gene_map = map_uniprot_simple(idlist)

    df = df.copy()
    df['Gene_Name'] = df[protein_id_col].map(gene_map) #take gene names we got from uniprot and add to dataframe

    total_rows = len(df)
    mapped_rows = df['Gene_Name'].notna().sum()
    unique_ids = df[protein_id_col].nunique()
    unique_mapped = df[df['Gene_Name'].notna()][protein_id_col].nunique()

    print(f"\n=== Mapping Summary ===")
    print(f"Total rows: {total_rows}")
    print(f"Rows with gene names: {mapped_rows} ({mapped_rows/total_rows*100:.1f}%)")
    print(f"Unique protein IDs: {unique_ids}")
    print(f"Unique IDs mapped: {unique_mapped} ({unique_mapped/unique_ids*100:.1f}%)")

    unmapped_unique = df[df['Gene_Name'].isna()][protein_id_col].unique()
    if len(unmapped_unique) > 0:
        print(f"\nUnmapped unique IDs ({len(unmapped_unique)}): {list(unmapped_unique[:20])}")

    return df


# NOW RUN THIS:
pddp = pd.read_table("bloodprotein_names.txt")
print("Original data:")
print(pddp.head())

# Create mapped dataframe
df_with_genes = create_mapped_dataframe(pddp)

# Check results
print("\n\nFirst 20 rows with gene names:")
print(df_with_genes.head(20))

print("\n\nSample of unique mappings:")
print(df_with_genes[['Protein_id', 'Gene_Name','MAPP_absoluteScore']].drop_duplicates().head(30))

#perm. unique
# df_unique = df_with_genes.drop_duplicates(subset=["Gene_Name", "MAPP_absoluteScore"], keep="first")
df_unique = df_with_genes.drop_duplicates(subset=["Gene_Name"], keep="first")

df_unique.to_csv("bloodprotein_with_genes.csv", index=False)
print("\n✓ Saved to bloodprotein_with_genes.csv")

goi = df_unique[['Gene_Name']]
print (goi)

#MAPP score above 5:
#filtering MAPP scores above 5 df_unique
filtered_df_unique = df_unique[df_unique['MAPP_absoluteScore'] >= 5]
filtered_df_unique.to_csv("5bloodprotein_with_genes.csv", index=False)
print("\n✓ Saved to bloodprotein_with_genes.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'bloodprotein_names.txt'

In [ ]:
# FOR A549 (same as above, just. few names changed)

import pandas as pd
import requests
import time
from typing import Dict, List

def clean_protein_ids(idlist: List[str]) -> List[str]:
    """Remove invalid IDs and get unique set"""
    valid_ids = []
    for uid in idlist:
        # Skip if empty, too short, doesn't start with letter, or is a decoy
        if (uid and
            len(uid) >= 6 and
            uid[0].isalpha() and
            uid != 'heavy' and
            not uid.startswith('REV__') and  # Filter out reverse decoys
            not uid.startswith('CON__')):    # Filter out contaminants
            valid_ids.append(uid)

    unique_ids = list(set(valid_ids))
    print(f"Cleaned: {len(idlist)} total -> {len(unique_ids)} unique valid IDs")
    return unique_ids

def map_uniprot_simple(idlist: List[str], batch_size: int = 50) -> Dict[str, str]:
    """
    Map UniProt IDs to gene names using the search endpoint.
    """
    mapping = {}
    unique_ids = clean_protein_ids(idlist)

    print(f"\nMapping {len(unique_ids)} unique UniProt IDs...")

    for i in range(0, len(unique_ids), batch_size):
        batch = unique_ids[i:i+batch_size]
        batch_num = i // batch_size + 1
        total_batches = (len(unique_ids) + batch_size - 1) // batch_size

        print(f"\nProcessing batch {batch_num}/{total_batches} ({len(batch)} IDs)...")

        # Use search endpoint with proper query format
        url = "https://rest.uniprot.org/uniprotkb/search"

        # Build query string for multiple accessions
        query = ' OR '.join([f'accession:{acc}' for acc in batch])

        params = {
            'query': query,
            'format': 'json',
            'fields': 'accession,gene_primary,gene_names',
            'size': batch_size
        }

        try:
            response = requests.get(url, params=params, timeout=30)

            if response.status_code == 200:
                data = response.json()

                if 'results' in data:
                    for entry in data['results']:
                        accession = entry.get('primaryAccession')

                        gene_name = None

                        # Method 1: Try genes array
                        if 'genes' in entry and entry['genes']:
                            gene_info = entry['genes'][0]
                            if 'geneName' in gene_info:
                                gene_name = gene_info['geneName'].get('value')

                        if accession and gene_name:
                            mapping[accession] = gene_name
                            print(f"  ✓ {accession} -> {gene_name}")

                mapped_in_batch = len([b for b in batch if b in mapping])
                print(f"  Mapped {mapped_in_batch}/{len(batch)} from this batch")
            else:
                print(f"  ⚠ Batch failed with status {response.status_code}")
                print(f"  Response: {response.text[:300]}")

            time.sleep(0.5)  # Be nice to the API

        except Exception as e:
            print(f"  ⚠ Error in batch {batch_num}: {e}")
            continue

    print(f"\n{'='*50}")
    print(f"Total mapped: {len(mapping)}/{len(unique_ids)}")
    print(f"{'='*50}")
    return mapping

def create_mapped_dataframe(df: pd.DataFrame,
                           protein_id_col: str = 'Protein_id',
                           score_col: str = 'MAPP_absoluteScore') -> pd.DataFrame:
    """Create a new dataframe with gene names added."""
    print(f"\nStarting mapping for column: {protein_id_col}")

    idlist = df[protein_id_col].tolist()
    gene_map = map_uniprot_simple(idlist)

    df = df.copy()
    df['Gene_Name'] = df[protein_id_col].map(gene_map)

    total_rows = len(df)
    mapped_rows = df['Gene_Name'].notna().sum()
    unique_ids = df[protein_id_col].nunique()
    unique_mapped = df[df['Gene_Name'].notna()][protein_id_col].nunique()

    print(f"\n{'='*50}")
    print(f"MAPPING SUMMARY")
    print(f"{'='*50}")
    print(f"Total rows: {total_rows}")
    print(f"Rows with gene names: {mapped_rows} ({mapped_rows/total_rows*100:.1f}%)")
    print(f"Unique protein IDs: {unique_ids}")
    print(f"Unique IDs mapped: {unique_mapped} ({unique_mapped/unique_ids*100:.1f}%)")

    unmapped_unique = df[df['Gene_Name'].isna()][protein_id_col].unique()
    if len(unmapped_unique) > 0:
        print(f"\nUnmapped unique IDs ({len(unmapped_unique)}):")
        print(list(unmapped_unique[:20]))

    return df


# MAIN EXECUTION
if __name__ == "__main__":
    print("Loading data...")
    pddp = pd.read_table("A549protein_names.txt")

    print("\nOriginal data:")
    print(pddp.head())
    print(f"\nShape: {pddp.shape}")
    print(f"Columns: {pddp.columns.tolist()}")

    # Check for decoy/contaminant entries
    decoy_count = pddp['Protein_id'].str.startswith('REV__').sum()
    contam_count = pddp['Protein_id'].str.startswith('CON__').sum()
    print(f"\nDecoy entries (REV__): {decoy_count}")
    print(f"Contaminant entries (CON__): {contam_count}")

    # Create mapped dataframe
    df_with_genes = create_mapped_dataframe(pddp)

    # Show results
    print("\n" + "="*50)
    print("RESULTS - First 20 rows")
    print("="*50)
    print(df_with_genes[['Protein_id', 'Gene_Name', 'MAPP_absoluteScore']].head(20))

    print("\n" + "="*50)
    print("UNIQUE MAPPINGS - Sample")
    print("="*50)
    print(df_with_genes[['Protein_id', 'Gene_Name', 'MAPP_absoluteScore']]
          .drop_duplicates(subset=['Protein_id'])
          .head(30))

    # Remove duplicates based on Gene_Name
    df_unique = df_with_genes.drop_duplicates(subset=["Gene_Name"], keep="first")

    # Save to CSV
    df_unique.to_csv("A549protein_with_genes.csv", index=False)
    print(f"\n✓ Saved {len(df_unique)} unique genes to A549protein_with_genes.csv")

    # Show gene names only
    goi = df_unique[['Gene_Name']].dropna()
    print(f"\nGenes of interest (first 20):")
    print(goi.head(20))

    #MAPP score above 5:
    #filtering MAPP scores above 5 df_unique
    filtered_df_unique = df_unique[df_unique['MAPP_absoluteScore'] >= 5]
    filtered_df_unique.to_csv("5bloodprotein_with_genes.csv", index=False)
    print("\n✓ Saved to bloodprotein_with_genes.csv")



Loading data...

Original data:
      A549_secretome                                    MAPP  \
0        GHQQLYWSHPR  GHQQLYWSHPRKFGQGSRSCRVCSNRHGLIRKYGLNMC   
1  GHQQLYWSHPRKFGQGS  GHQQLYWSHPRKFGQGSRSCRVCSNRHGLIRKYGLNMC   
2        GHQQLYWSHPR   GHQQLYWSHPRKFGQGSRSCRVCSNRHGLIRKYGLNM   
3  GHQQLYWSHPRKFGQGS   GHQQLYWSHPRKFGQGSRSCRVCSNRHGLIRKYGLNM   
4        GHQQLYWSHPR     GHQQLYWSHPRKFGQGSRSCRVCSNRHGLIRKYGL   

   MAPP_absoluteScore Protein_id  
0            7.499082     P62273  
1            7.499082     P62273  
2            7.460637     P62273  
3            7.460637     P62273  
4            7.310881     P62273  

Shape: (3122, 4)
Columns: ['A549_secretome', 'MAPP', 'MAPP_absoluteScore', 'Protein_id']

Decoy entries (REV__): 5
Contaminant entries (CON__): 0

Starting mapping for column: Protein_id
Cleaned: 3122 total -> 134 unique valid IDs

Mapping 134 unique UniProt IDs...

Processing batch 1/3 (50 IDs)...
  ✓ O95139 -> NDUFB6
  ✓ P05204 -> HMGN2
  ✓ P08670 -> VIM
  ✓ P10619 ->


**DATA** **WRANGLING**

In [6]:
#reading in bone dataset
control_1 = pd.read_excel("GSM5073659_Ctrl1_3D.xlsx")
control_2 = pd.read_excel("GSM5073660_Ctrl2_3D.xlsx")
control_3 = pd.read_excel("GSM5073661_Ctrl3_3D.xlsx")
control_4 = pd.read_excel("GSM5073665_Ctrl1_14D.xlsx")
control_5 = pd.read_excel("GSM5073666_Ctrl2_14D.xlsx")
control_6 = pd.read_excel("GSM5073667_Ctrl3_14D.xlsx")

infected_1 = pd.read_excel("GSM5073662_IAOM1_3D.xlsx")
infected_2 = pd.read_excel("GSM5073663_IAOM2_3D.xlsx")
infected_3 = pd.read_excel("GSM5073664_IAOM3_3D.xlsx")
infected_4 = pd.read_excel("GSM5073668_IAOM1_14D.xlsx")
infected_5 = pd.read_excel("GSM5073669_IAOM2_14D.xlsx")
infected_6 = pd.read_excel("GSM5073670_IAOM3_14D.xlsx")


In [39]:
#combine all bone data into one dataframe:
#we're using fpkm, remove counts
#new data frame: for gene_name & fpkm


dfs = [control_1, control_2, control_3,
      control_4, control_5, control_6,
      infected_1, infected_2, infected_3,
      infected_4, infected_5, infected_6]

merged_df = dfs[0]

for df in dfs[1:]:
  merged_df = merged_df.merge(
      df,
      on = ["gene_id", "gene_name"],
      how = "inner"
  )
print(type(merged_df))



#renaming columns


merged_df = merged_df.rename(columns = {
    "Con1_3D_count": "Ctrl_BM1_count",
    "Con1_3D_fpkm": "Ctrl_BM1_fpkm",
    "Con2_3D_count": "Ctrl_BM2_count",
    "Con2_3D_fpkm": "Ctrl_BM2_fpkm",
    "Con3_3D_count": "Ctrl_BM3_count",
    "Con3_3D_fpkm": "Ctrl_BM3_fpkm",
    "Con1_14D_count": "Ctrl_BM4_count",
    "Con1_14D_fpkm": "Ctrl_BM4_fpkm",
    "Con2_14D_count":"Ctrl_BM5_count",
    "Con2_14D_fpkm":"Ctrl_BM5_fpkm",
    "Con3_14D_count":"Ctrl_BM6_count",
    "Con3_14D_fpkm":"Ctrl_BM6_fpkm",
    "Inf2_3D_count": "Inf_BM2_count",
    "Inf2_3D_fpkm": "Inf_BM2_fpkm",
    "Inf3_3D_count": "Inf_BM3_count",
    "Inf3_3D_fpkm": "Inf_BM3_fpkm",
    "Inf1_14D_count": "Inf_BM4_count",
    "Inf1_14D_fpkm": "Inf_BM4_fpkm",
    "Inf2_14D_count": "Inf_BM5_count",
    "Inf2_14D_fpkm": "Inf_BM5_fpkm",
    "Inf3_14D_count": "Inf_BM6_count",
    "Inf3_14D_fpkm": "Inf_BM6_fpkm"


})

#merged_df = merged_df.set_index("gene_id")



merged_df.head()


<class 'pandas.core.frame.DataFrame'>


,gene_id,gene_name,Ctrl_BM1_count,Ctrl_BM1_fpkm,Ctrl_BM2_count,Ctrl_BM2_fpkm,Ctrl_BM3_count,Ctrl_BM3_fpkm,Ctrl_BM4_count,Ctrl_BM4_fpkm,...,Inf_BM2_count,Inf_BM2_fpkm,Inf_BM3_count,Inf_BM3_fpkm,Inf_BM4_count,Inf_BM4_fpkm,Inf_BM5_count,Inf_BM5_fpkm,Inf_BM6_count,Inf_BM6_fpkm
0,ENSMUSG00000056071,S100a9,162642,17337.041178,214016,22942.990774,169850,18292.978420,154461,16656.975474,...,183549,16383.297157,184734,16422.147542,246357,25933.736248,167028,17578.099432,221064,24290.859853
1,ENSMUSG00000064351,mt-Co1,135839,4892.251438,124116,4495.454203,118699,4319.246105,127324,4639.055809,...,154269,4652.324180,161513,4851.015015,151601,5391.930281,182144,6476.487399,147408,5472.527308
2,ENSMUSG00000032484,Ngp,131047,5967.173355,156639,7173.035607,127052,5845.204014,114701,5283.768792,...,120445,4592.375156,146654,5568.988559,169631,7627.894929,106341,4780.602926,151392,7106.030837
3,ENSMUSG00000056054,S100a8,127175,10196.592266,147815,11918.821837,119135,9650.932376,99379,8060.889336,...,131392,8821.231947,117080,7828.469465,179141,14184.244525,118500,9380.198283,152016,12563.921019
4,ENSMUSG00000069516,Lyz2,100193,4236.373755,114564,4871.543334,96148,4107.463750,80542,3445.199119,...,109387,3872.840499,106339,3749.646284,154086,6433.954388,104237,4351.301268,127101,5539.728052


In [ ]:
#heat map
#combine a
#input: RNA seq data








In [ ]:
#blood2 = pd.read_table("GSE127813_Whole_blood_gene_expression_matrix(1).txt")
#print(blood2)
# print(blood2.shape)
# print(blood2.columns)
# print(blood2.r)

In [ ]:
!pip install pydeseq2
!pip install matplotlib scikit-learn pandas
import os
import pickle as pkl
import numpy as np


VOLCANO

In [40]:
#volcano plot input manipulation
volcano_df = merged_df.loc[:, ~merged_df.columns.str.contains("fpkm")]
volcano_df1 = volcano_df.loc[:, ~volcano_df.columns.str.contains("gene_id")]
#switching columns and rows
volcano_df_switched = volcano_df1.transpose()
volcano_df_switched.head(5)

#making metadata dataset
#metadata_df = pd.DataFrame({
 #  "condition": [],
  #  "condition": [""]
#})

metadata_df = pd.DataFrame(index = volcano_df_switched.index, columns = ["", ""])








,0,1,2,3,4,5,6,7,8,9,...,54522,54523,54524,54525,54526,54527,54528,54529,54530,54531
gene_name,S100a9,mt-Co1,Ngp,S100a8,Lyz2,Eef1a1,Ltf,Hba-a1,Col1a2,Col1a1,...,Vmn1r186,AC102264.1,AC234645.1,AC168977.2,RF00004,CR524822.1,CAAA01205117.2,CAAA01205117.1,CAAA01064564.1,Vmn2r122
Ctrl_BM1_count,162642,135839,131047,127175,100193,113438,82491,102220,68039,63533,...,0,0,0,0,0,0,0,0,0,0
Ctrl_BM2_count,214016,124116,156639,147815,114564,105553,96262,98996,60277,56394,...,0,0,0,0,0,0,0,0,0,0
Ctrl_BM3_count,169850,118699,127052,119135,96148,104237,81879,105341,99775,93664,...,0,0,0,0,0,0,0,0,0,0
Ctrl_BM4_count,154461,127324,114701,99379,80542,100281,66467,121978,52321,43705,...,0,0,0,0,0,0,0,0,0,0
